In [1]:
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.extensions import UnitaryGate
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import MCXGate

In [2]:
def reflection_operator(n):
    qc1 = QuantumCircuit(n)
    qc1.x(range(n))
    qc1.h(0)
    qc1.append(MCXGate(n-1), list(range(1, n))+[0])
    qc1.h(0)
    qc1.x(range(n))
    return qc1.to_gate()

def grover_gate(n, oracle_gate):
    qc = QuantumCircuit(n)
    qc.h(range(n))
    qc.append(oracle_gate, range(n))
    qc.h(range(n))
    qc.append(reflection_operator(n), range(n))
    qc.h(range(n))
    return qc.to_gate()

In [3]:
n = 4
N = 2**n
oracle_vals = np.array([1.]*N)
oracle_vals[10] = -1.
oracle_gate = UnitaryGate(np.diag(oracle_vals))

In [4]:
qc = QuantumCircuit(n, n)
qc.append(grover_gate(n, oracle_gate), range(n))
qc.measure(range(n), range(n))

In [5]:
qc.draw()

┌─────────────┐┌─┐         
q_0: ┤0            ├┤M├─────────
     │             │└╥┘┌─┐      
q_1: ┤1            ├─╫─┤M├──────
     │  circuit-88 │ ║ └╥┘┌─┐   
q_2: ┤2            ├─╫──╫─┤M├───
     │             │ ║  ║ └╥┘┌─┐
q_3: ┤3            ├─╫──╫──╫─┤M├
     └─────────────┘ ║  ║  ║ └╥┘
c: 4/════════════════╩══╩══╩══╩═
                     0  1  2  3

In [6]:
backend = Aer.get_backend('aer_simulator')
tqc = transpile(qc, backend)
job = backend.run(tqc, shots=1000)
result = job.result()


In [7]:
result.get_counts()

{'1110': 43,
 '0101': 33,
 '0010': 32,
 '0100': 38,
 '1100': 39,
 '1000': 40,
 '0000': 43,
 '1001': 38,
 '1011': 41,
 '0001': 32,
 '0011': 27,
 '0111': 32,
 '1101': 25,
 '1111': 27,
 '0110': 26,
 '1010': 484}